# Perturbation with Dynare-like syntax

Consider the following way to define a model.

- improve model printing
- change model definition so that it represents rbc rather than neoclassical
- study source code of minidyn.jl
    - model structure
    - static arrays
    - macro
- add solution method
    




In [14]:
include("minidyn.jl")

steady_state (generic function with 1 method)

In [9]:
model = let  

    α = 0.3
    β = 0.96
    γ = 4.0
    δ = 0.1
    
    k = ((1/β-(1-δ))/α)^(1/(α-1))
    z = 0.0

    i = δ*k

    y = k^α
    c = y-i
    e = 0.0

    s = SLVector( (;k,z) )
    x = SLVector( (;i,c) )
    e = SLVector( (;e,) )
    p = SLVector( (;α, β, γ, δ) )

    PModel(s,x,e,p)

end

PModel{(:k, :z), (:i, :c), (:e,), (:α, :β, :γ, :δ), 2, 2, 1, 4}(2-element SLArray{Tuple{2}, Float64, 1, 2, (:k, :z)} with indices SOneTo(2):
 :k => 2.920822149964071
 :z => 0.0, 2-element SLArray{Tuple{2}, Float64, 1, 2, (:i, :c)} with indices SOneTo(2):
 :i => 0.29208221499640713
 :c => 1.087194911375516, 1-element SLArray{Tuple{1}, Float64, 1, 1, (:e,)} with indices SOneTo(1):
 :e => 0.0, 4-element SLArray{Tuple{4}, Float64, 1, 4, (:α, :β, :γ, :δ)} with indices SOneTo(4):
 :α => 0.3
 :β => 0.96
 :γ => 4.0
 :δ => 0.1)

In [10]:
@dyneq function transition(model)

    k[t] = k[t-1]*(1-δ) + i[t-1]
    z[t] = 0.7* z[t-1]

end

transition (generic function with 3 methods)

In [11]:
@dyneq function arbitrage(model)

    c[t] == exp(z[t])*k[t]^α - i[t]
    β*(c[t+1]/c[t])^(-γ)* ( (1-δ) + α*exp(z[t+1])*k[t+1]^(α-1)) == 1

end

arbitrage (generic function with 3 methods)

In [12]:
ss = steady_state(model)

(s = 2-element SLArray{Tuple{2}, Float64, 1, 2, (:k, :z)} with indices SOneTo(2):
 :k => 2.920822149964071
 :z => 0.0, x = 2-element SLArray{Tuple{2}, Float64, 1, 2, (:i, :c)} with indices SOneTo(2):
 :i => 0.29208221499640713
 :c => 1.087194911375516)

In [17]:
pm = perturb(model)

(A = [0.14166666666666672 1.379277126371923; 0.0 0.0], B = [-1.0 -1.0; -0.0 -3.679193085018409], C = [0.0 0.0; 0.03259356274094644 -0.13600000000000004], D = [0.0 0.0; 0.0 3.679193085018409], E = [0.9 0.0; 0.0 0.7], F = [1.0 0.0; 0.0 0.0], Q = [0.0; 0.0;;])

In [25]:
# .....